# **Citation Generation**

In [1]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00


In [2]:
from langchain_core.documents import Document
from langchain.retrievers import EnsembleRetriever # Supports Ensembling of results from multiple retrievers
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
from pydantic import BaseModel, Field
from typing import List
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
from langchain import PromptTemplate
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import os
import json
from google.colab import files
import time
from langchain_groq import ChatGroq

## **Method Type 1: Using Pydantic schema for validation with models that support tool/function calling functionality and JSON modes i.e. provide native APIs for structuring outputs**

### **User Action Required**

To utilise method type 1, we leverage the the Groq free API that was mentioned in a list of [examples/LLM providers](https://python.langchain.com/v0.1/docs/modules/model_io/chat/structured_output/) that LangChain provided that supported function calling and JSON

To set up your colab environment for Groq
1. Go to ```https://console.groq.com/keys``` to create an API key
2. Add the created API key to your google colab secrets with the name ```GROQ_API_KEY```

<u>Pydantic Objects</u>

Leverage Pydantic objects to validate that data conforms to the output we expect

Beyond just the structure of the Pydantic class, the name of the Pydantic class, the docstring, and the names and provided descriptions of parameters are very important

Define the fields needed and the corresponding types
Note that the docstrings here are crucial, as they will be passed along
to the model along with the class name.

<u>```.with_structured_output()```</u>
Use LangChain's with_structured_output. Takes in schema (Pydantic class, JSON etc.) that specifies the structured
output. Returns an object corresponding to the provided schema
The method will add the necessary model arguments and output parsers to get the structured output

tool/function calling: model comes up with arguments to a tool

with_structured_output uses a mode's function/tool calling API

Leverages the model's native tools and functions that it can call to ensure it conforms to the
schema

- https://python.langchain.com/v0.1/docs/modules/model_io/chat/structured_output/
- https://python.langchain.com/docs/how_to/structured_output/
- https://python.langchain.com/v0.1/docs/use_cases/question_answering/citations/
- https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/

In [ ]:
# List of models Groq provides: https://console.groq.com/docs/models
# https://python.langchain.com/docs/integrations/chat/groq/
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
llm = ChatGroq()

### **Simple Experiment Data to test and observe behaviour of Method Type 1**

In [ ]:
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'},
    ),
    Document(
        page_content="The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembes a tongue.",
        metadata={'country': 'Norway', 'source': 'norwayhikes', 'link': 'https://www.norwayhikes.com/'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    )
]

### **Test Question**

In [ ]:
question = "what are the best hikes in norway?"

In [ ]:
prompt = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer from the retrieved context, just say that you don't know.
Question: {question}
Context: {context}
Helpful Answer:
"""

### **Test Citation with source**

In [ ]:
class CitedAnswer(BaseModel):
    """Answer the user question based only on the given sources, and cite the sources used."""

    answer: str = Field(
        ...,
        description="The answer to the user question, which is based only on the given sources.",
    )
    citations: List[str] = Field(
        ...,
        description="The source in the Document metadata which justify the answer.",
    )

structured_llm = llm.with_structured_output(CitedAnswer)

In [ ]:
answer = structured_llm.invoke(prompt.format(question=question,context=docs))

In [ ]:
answer

CitedAnswer(answer='The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga.', citations=['visitNorway', 'norwayhikes'])

In [ ]:
answer.answer

'The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga.'

In [ ]:
answer.citations

['visitNorway', 'norwayhikes']

### **Test Citation with link**

In [ ]:
class CitedAnswer(BaseModel):
    """Answer the user question based only on the given sources, and cite the sources used."""

    answer: str = Field(
        ...,
        description="The answer to the user question, which is based only on the given sources.",
    )
    citations: List[str] = Field(
        ...,
        description="The link in the Document metadata which justify the answer.",
    )

structured_llm = llm.with_structured_output(CitedAnswer)

In [ ]:
answer = structured_llm.invoke(prompt.format(question=question,context=docs))

In [ ]:
answer

CitedAnswer(answer='The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga.', citations=['https://www.visitnorway.com/', 'https://www.norwayhikes.com/'])

In [ ]:
answer.answer

'The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga.'

In [ ]:
answer.citations

['https://www.visitnorway.com/', 'https://www.norwayhikes.com/']

### **Test Refusal**

In [ ]:
question = "what is the speed of a rocket"
answer = structured_llm.invoke(prompt.format(question=question,context=docs))

In [ ]:
answer

CitedAnswer(answer="I don't have information about the speed of a rocket from the provided context.", citations=[])

**TODO if got time**

- Few-shot prompting
  - https://python.langchain.com/docs/how_to/structured_output/
  - For citations with source and link? But maybe that's more relevant to method type 2?
- Fallback to raw outputs
  - https://python.langchain.com/docs/how_to/structured_output/


<br/>
<br/>
<br/>

## **Method Type 2: Direct Prompting**

Not all models support tool calling/function calling or have native JSON mode support. This method explores the use of direct prompting to ask the model to use a specific format

Additionally, we use few-shot prompting to enable in-context learning

### **Simple Experiment Data to test and observe behaviour of Method Type 2**

In [ ]:
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'},
    ),
    Document(
        page_content="The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembes a tongue.",
        metadata={'country': 'Norway', 'source': 'norwayhikes', 'link': 'https://www.norwayhikes.com/'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    )
]

### **Question**

In [ ]:
question = "what are the best hikes in norway?"

In [ ]:
prompt = '''You are a helpful assistant for answering questions using the provided retrieved documents.

Use only the provided context to generate your answer. If the answer cannot be found in the context, say "I don't know."

Respond in the following JSON format:
{{
    "answer": "<Your answer here>", "citations": ["<Link1>", "<Link2>", ...]  // Use the metadata links of the documents that support your answer.
}}

Here are some examples:

{{
    "answer": "The best waterfall in iceland is the Skogafoss waterfall and Gullfoss Waterfall", "citations": ["https://www.besticelandwaterfalls.com", "https://www.exploreiceland.com"]
}}

{{
    "answer": "Transportation in Norway is very easy with its public bus service called Omio", "citations": ["https://www.omio.com", "https://www.visitnorway.com"]
}}

Question: {question}

Context: {context}

Helpful Answer:
'''

### **Test with smaller sized model**

In [ ]:
llm_small = HuggingFacePipeline(
      pipeline=pipeline(
        model="Qwen/Qwen2.5-3B-Instruct",
        task="text-generation",
        temperature=0.2,
        do_sample=True,
        repetition_penalty=1.1,
        max_new_tokens=400,
        device_map="auto"
      )
    )

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
small_sized_llm_pipeline = llm_small.bind(skip_prompt=True) | StrOutputParser()
answer = small_sized_llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [ ]:
answer

'{\n    "answer": "The best hikes in Norway include the Reinebringen hike in the Lofoten Islands, Preikestolen, Kjeragbolten, and Trolltunga.", "citations": ["https://www.norwayhikes.com/", "https://www.visitnorway.com/"]\n}\n{\n    "answer": "I don\'t know.",\n    "citations": []\n} {\n    "answer": "The best hikes in Norway include the Reinebringen hike in the Lofoten Islands, Preikestolen, Kjeragbolten, and Trolltunga.", "citations": ["https://www.norwayhikes.com/", "https://www.visitnorway.com/"]\n}'

**Testing refusal with smaller sized model**

In [ ]:
question = "what is the speed of a rocket"

In [ ]:
answer = small_sized_llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [ ]:
answer

'{\n    "answer": "I don\'t know.", "citations": []\n}\nExplanation for the helper answer:\nNone of the provided documents contain any information about the speed of rockets or transportation methods in different countries. The content discusses hiking spots in Norway, popular foods in Iceland, and transportation options within Reykjavik. There is no relevant data available to answer the question about the speed of a rocket.\n\n```json\n{\n    "answer": "I don\'t know.", "citations": []\n}\n```'

### **Test with larger sized model**

In [ ]:
llm_large = ChatGroq()
larger_sized_llm_pipeline = llm_large | StrOutputParser()
answer = larger_sized_llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [ ]:
answer

'{\n    "answer": "The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga.",\n    "citations": ["https://www.visitnorway.com", "https://www.norwayhikes.com/"]\n}'

In [ ]:
json.loads(answer)

{'answer': 'The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga.',
 'citations': ['https://www.visitnorway.com', 'https://www.norwayhikes.com/']}

**Testing refusal with larger sized model**

In [ ]:
question = "what is the speed of a rocket"

In [ ]:
answer = larger_sized_llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [ ]:
answer

"I don't know the speed of a rocket because the provided context does not include any information about rockets or their speeds."

<br/>

**As we can see, the smaller sized model does not generate as well-strucutred responses as compared to the larger sized model**

**TODO**

Can potentially do a runnableparallel to add the retrieved docs to the result as the 'citations'
- https://python.langchain.com/v0.1/docs/use_cases/question_answering/citations/#retrieval-post-processing

<br/>
<br/>
<br/>

## **Method Type 3: Direct Prompting with statement-wise citations**

Not all models support tool calling/function calling or have native JSON mode support. This method explores the use of direct prompting to ask the model to use a specific format

Additionally, we use few-shot prompting to enable in-context learning

### **Simple Experiment Data to test and observe behaviour of Method Type 3**

In [ ]:
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'},
    ),
    Document(
        page_content="The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembes a tongue.",
        metadata={'country': 'Norway', 'source': 'norwayhikes', 'link': 'https://www.norwayhikes.com/'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'},
    ),
    Document(
        page_content="Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!",
        metadata={'country': 'Iceland', 'source': 'IcelandGov', 'link': 'https://www.welcometoiceland.com/'},
    ),
    Document(
        page_content="The pasteries and bread in Iceland are fantastic, there are many bakeries in Iceland. One of the most popular bread is called dark rye bread ",
        metadata={'country': 'Iceland', 'source': 'IcelandFood', 'link': 'https://www.icelandicdelicacies.com/'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    )
]

### **Question**

In [ ]:
question = "what is the best food in iceland?"

In [ ]:
prompt = """
You are an helpful assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
For each relevant statement in your answer, provide it as a separate JSON object, specifying the citations that support only that statement.
If the answer cannot be found in the context, just say that you don't know and do not cite irrelevant sources.
Make the statements sound coherent together.

Respond in the following JSON format:
[
    {{
        "answer": "<Statement 1>",
        "citations": ["<Link1>", "<Link2>", ...] // Use the metadata links of the documents that support the statement
    }},
    {{
        "answer": "<Statement 2>",
        "citations": ["<Link3>", "<Link4>", ...]
    }}
]

Here is an example:

[
    {{
        "answer": "The best hikes in Norway include the Reinebringen hike in the Lofoten islands.",
        "citations": ["https://www.norwayhikes.com/"]
    }},
    {{
        "answer": "At its peak, you can admire the beautiful view of Reine town as well as the breathtaking fjords that surround it.",
        "citations": ["https://www.norwayhikes.com/"]
    }},
    {{
        "answer": "Another famous hike that is a must-go in Norway is Trolltunga!",
        "citations": ["https://www.visitnorway.com/"]
    }},
    {{
        "answer": "Trolltunga in english is called 'Troll's Tongue' as it resembles a tongue and the Norweigens believe that the mountain was once a Troll.",
        "citations": ["https://www.visitnorway.com/", "https://www.norwaymyths.com/"]
    }},
    {{
        "answer": "The best season to enjoy these amazing hikes would be from May to August where it's not too hot yet and too cold, however safety is always paramount and it is important to bring the right gear for hiking and check safety advisories.",
        "citations": ["https://www.norwayadventrues.com/","https://www.norwaygov.com/"]
    }}
]

Question: {question}

Context: {context}

Helpful Answer:
"""

In [ ]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
llm = ChatGroq()
llm_pipeline = llm | StrOutputParser()
answer = llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [ ]:
answer

'[\n    {\n        "answer": "A famous street food in Iceland is the Hotdog, called the Baejarins Beztu Pylsur hot dog and made of a mix of lamb, beef, and pork.",\n        "citations": ["https://www.icelandtours.com/"]\n    },\n    {\n        "answer": "Iceland is also known for its fresh fish, with dishes like fish and chips, arctic cod, and salmon soup being popular.",\n        "citations": ["https://www.welcometoiceland.com/"]\n    },\n    {\n        "answer": "Other delicacies in Iceland include pastries and bread, with dark rye bread being one of the most popular types.",\n        "citations": ["https://www.icelandicdelicacies.com/"]\n    }\n]'

In [ ]:
json.loads(answer)

[{'answer': 'A famous street food in Iceland is the Hotdog, called the Baejarins Beztu Pylsur hot dog and made of a mix of lamb, beef, and pork.',
  'citations': ['https://www.icelandtours.com/']},
 {'answer': 'Iceland is also known for its fresh fish, with dishes like fish and chips, arctic cod, and salmon soup being popular.',
  'citations': ['https://www.welcometoiceland.com/']},
 {'answer': 'Other delicacies in Iceland include pastries and bread, with dark rye bread being one of the most popular types.',
  'citations': ['https://www.icelandicdelicacies.com/']}]

<br/>
<br/>
<br/>

## **Method Type 4: Direct Prompting with statement-wise citations**

Not all models support tool calling/function calling or have native JSON mode support. This method explores the use of direct prompting to ask the model to use a specific format

Additionally, we use few-shot prompting to enable in-context learning

### **Simple Experiment Data to test and observe behaviour of Method Type 4**

In [3]:
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'},
    ),
    Document(
        page_content="The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembes a tongue.",
        metadata={'country': 'Norway', 'source': 'norwayhikes', 'link': 'https://www.norwayhikes.com/'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'},
    ),
    Document(
        page_content="Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!",
        metadata={'country': 'Iceland', 'source': 'IcelandGov', 'link': 'https://www.welcometoiceland.com/'},
    ),
    Document(
        page_content="The pasteries and bread in Iceland are fantastic, there are many bakeries in Iceland. One of the most popular bread is called dark rye bread ",
        metadata={'country': 'Iceland', 'source': 'IcelandFood', 'link': 'https://www.icelandicdelicacies.com/'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    )
]

### **Question**

In [12]:
question = "what are the best hikes in Norway?"

In [13]:
prompt = template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
For each statement, include inline numbered citations (e.g., [1], [2]) for the Document Objects supporting it.
At the bottom, provide the full citations corresponding to each number.

If the answer cannot be found in the context, say "I don't know." Do not make up information.

Respond in the following format:
---
Statement 1 [1]. Statement 2 [2, 3].

Citations:
[1] <Full Citation 1> //Document Object
[2] <Full Citation 2>
[3] <Full Citation 3>
---

Here is an example:

---
The best food in Iceland is the Baejarins Beztu Pylsur hot dog, which is made of a mix of lamb, beef, and pork [1]. Other famous delicacies include Fish and Chips as well as the classic fish and chips dish [1, 2]. Additionally, Iceland is very famous for its freshly caught fish from the arctic sea [2].

Citations
[1]: Document(page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",metadata={{'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'}})
[2]: Document(page_content="Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!",metadata={{'country': 'Iceland', 'source': 'IcelandGov', 'link': 'https://www.welcometoiceland.com/'}})
---

Question: {question}

Context: {context}

Helpful Answer:
"""

In [14]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
llm = ChatGroq()
llm_pipeline = llm | StrOutputParser()
answer = llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [15]:
answer

"The best hikes in Norway are the Reinebringen hike in the Lofoten islands [1], and Preikestolen, Kjeragbolten, and Trolltunga [2]. The Reinebringen hike is 448 meters high with a challenging steep trail, offering an iconic view from the summit of Reine [1]. Preikestolen, Kjeragbolten, and Trolltunga are famous hikes in Norway, featuring a beautiful fjord, a famous boulder stuck between a mountain crevasse, and a tongue-shaped viewpoint, respectively [2].\n\nCitations:\n[1] Document(metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.')\n[2] Document(metadata={'country': 'No

In [16]:
answer.split('\n')

['The best hikes in Norway are the Reinebringen hike in the Lofoten islands [1], and Preikestolen, Kjeragbolten, and Trolltunga [2]. The Reinebringen hike is 448 meters high with a challenging steep trail, offering an iconic view from the summit of Reine [1]. Preikestolen, Kjeragbolten, and Trolltunga are famous hikes in Norway, featuring a beautiful fjord, a famous boulder stuck between a mountain crevasse, and a tongue-shaped viewpoint, respectively [2].',
 '',
 'Citations:',
 "[1] Document(metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.')",
 "[2] Document(metadata={'

<br/>
<br/>
<br/>

## **Method Type 5: Direct Prompting with statement-wise citations**

Not all models support tool calling/function calling or have native JSON mode support. This method explores the use of direct prompting to ask the model to use a specific format

Additionally, we use few-shot prompting to enable in-context learning

### **Simple Experiment Data to test and observe behaviour of Method Type 5**

In [76]:
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'},
    ),
    Document(
        page_content="The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembes a tongue.",
        metadata={'country': 'Norway', 'source': 'norwayhikes', 'link': 'https://www.norwayhikes.com/'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'},
    ),
    Document(
        page_content="Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!",
        metadata={'country': 'Iceland', 'source': 'IcelandGov', 'link': 'https://www.welcometoiceland.com/'},
    ),
    Document(
        page_content="The pasteries and bread in Iceland are fantastic, there are many bakeries in Iceland. One of the most popular bread is called dark rye bread ",
        metadata={'country': 'Iceland', 'source': 'IcelandFood', 'link': 'https://www.icelandicdelicacies.com/'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    ),
    Document(
        page_content="Driving in Iceland is an amazing experience - open roads, majestic volcanos and towering mountains along the way, sheep and arctic foxes make it a great experience. All you need is an international driving license. And, please drive slowly during the winter!",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    ),
    Document(
        page_content="Iceland is a must-go to place for adventurous people! You can hike active volcanoes, drive a jeep through the volcanic ash, explore a natural ice cave, see waterfalls. There are so many opportunities for an adventurer.",
        metadata={'country': 'Iceland', 'source': 'IcelandAdventures', 'link': 'https://www.icelandadventures.com/'},
    ),
    Document(
        page_content="One of the most famous diving sites in the world, Silfra, is located in Iceland! It is the only diving site in the world where you can dive between 2 tectonic plates. The water is also so fresh that you can drink from it, it is the best water that you will ever taste.",
        metadata={'country': 'Iceland', 'source': 'IcelandDiving', 'link': 'https://www.icelanddiving.com/'},
    ),
]

### **Question**

In [77]:
question = "What can i do in iceland?"

In [78]:
prompt = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
For each statement, include inline numbered citations (e.g., [1], [2]) for the Document Objects supporting it.
Ensure **every statement** in your answer has at least one inline numbered citation.
Statements without sufficient support from the context should not be included in the answer.
At the bottom, provide the full citations corresponding to each number.

Write your response in a natural and coherent way, ensuring that the statements flow logically and transition smoothly between ideas. Use connecting words and phrases (e.g., "Additionally," "Furthermore," "For instance," "As a result,") to enhance readability.

If the answer cannot be found in the context, say "I don't know." Do not make up information.

Respond in the following format:
---
Statement 1 [1]. Statement 2 [2, 3].

Citations:
[1] <Full Citation 1> //Document Object
[2] <Full Citation 2>
[3] <Full Citation 3>
---

Here is an example:

---
The best food in Iceland is the Baejarins Beztu Pylsur hot dog, which is made of a mix of lamb, beef, and pork [1]. Other famous delicacies include Fish and Chips as well as the classic fish and chips dish [1, 2]. Additionally, Iceland is very famous for its freshly caught fish from the arctic sea [2].

Citations
[1]: Document(page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",metadata={{'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'}})
[2]: Document(page_content="Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!",metadata={{'country': 'Iceland', 'source': 'IcelandGov', 'link': 'https://www.welcometoiceland.com/'}})
---

Question: {question}

Context: {context}

Helpful Answer:
"""

In [79]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
llm = ChatGroq()
llm_pipeline = llm | StrOutputParser()
answer = llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [80]:
answer

'In Iceland, you can indulge in the famous street food, the Baejarins Beztu Pylsur hot dog, which is made of a mix of lamb, beef, and pork [4]. Other culinary delights include Fish and Chips and Tommi\'s burger [4]. Additionally, Iceland is renowned for its freshly caught fish from the arctic sea, with dishes like the classic fish and chips, arctic cod, and salmon soup being particularly popular [5].\n\nIf you\'re an adventurous person, Iceland offers numerous opportunities. You can hike active volcanoes, drive a jeep through the volcanic ash, explore a natural ice cave, and see waterfalls [6]. Moreover, Iceland is home to one of the most famous diving sites in the world, Silfra, where you can dive between two tectonic plates and drink the fresh, clean water [7].\n\nFor transportation within Reykjavik, a public bus service called BSI is readily available. You just need to download their mobile app and follow the instructions [8]. However, for traveling outside Reykjavik, a car is requi

In [81]:
answer.split('\n')

["In Iceland, you can indulge in the famous street food, the Baejarins Beztu Pylsur hot dog, which is made of a mix of lamb, beef, and pork [4]. Other culinary delights include Fish and Chips and Tommi's burger [4]. Additionally, Iceland is renowned for its freshly caught fish from the arctic sea, with dishes like the classic fish and chips, arctic cod, and salmon soup being particularly popular [5].",
 '',
 "If you're an adventurous person, Iceland offers numerous opportunities. You can hike active volcanoes, drive a jeep through the volcanic ash, explore a natural ice cave, and see waterfalls [6]. Moreover, Iceland is home to one of the most famous diving sites in the world, Silfra, where you can dive between two tectonic plates and drink the fresh, clean water [7].",
 '',
 'For transportation within Reykjavik, a public bus service called BSI is readily available. You just need to download their mobile app and follow the instructions [8]. However, for traveling outside Reykjavik, a c

<br/>
<br/>
<br/>

## **Method Type 6: Direct Prompting with statement-wise citations**

Not all models support tool calling/function calling or have native JSON mode support. This method explores the use of direct prompting to ask the model to use a specific format

Additionally, we use few-shot prompting to enable in-context learning

### **Simple Experiment Data to test and observe behaviour of Method Type 6**

In [129]:
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'},
    ),
    Document(
        page_content="The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembes a tongue.",
        metadata={'country': 'Norway', 'source': 'norwayhikes', 'link': 'https://www.norwayhikes.com/'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'},
    ),
    Document(
        page_content="Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!",
        metadata={'country': 'Iceland', 'source': 'IcelandGov', 'link': 'https://www.welcometoiceland.com/'},
    ),
    Document(
        page_content="The pasteries and bread in Iceland are fantastic, there are many bakeries in Iceland. One of the most popular bread is called dark rye bread ",
        metadata={'country': 'Iceland', 'source': 'IcelandFood', 'link': 'https://www.icelandicdelicacies.com/'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    ),
    Document(
        page_content="Driving in Iceland is an amazing experience - open roads, majestic volcanos and towering mountains along the way, sheep and arctic foxes make it a great experience. All you need is an international driving license. And, please drive slowly during the winter!",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    ),
    Document(
        page_content="Iceland is a must-go to place for adventurous people! You can hike active volcanoes, drive a jeep through the volcanic ash, explore a natural ice cave, see waterfalls. There are so many opportunities for an adventurer.",
        metadata={'country': 'Iceland', 'source': 'IcelandAdventures', 'link': 'https://www.icelandadventures.com/'},
    ),
    Document(
        page_content="One of the most famous diving sites in the world, Silfra, is located in Iceland! It is the only diving site in the world where you can dive between 2 tectonic plates. The water is also so fresh that you can drink from it, it is the best water that you will ever taste.",
        metadata={'country': 'Iceland', 'source': 'IcelandDiving', 'link': 'https://www.icelanddiving.com/'},
    ),
]

### **Question**

In [130]:
question = "What can i do in iceland?"

In [131]:
prompt = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
For each statement in your answer, **you must include at least one inline numbered citation** (e.g., [1], [2]) for the Document Objects supporting it.
Do not include any statements that lack sufficient support from the provided context.

Ensure the numbering of citations starts from **1** for each response.
At the bottom, provide the full citations corresponding to each number, but include **only the page content** of the Document Object (exclude metadata).

Write your response in a natural and coherent way, ensuring that the statements flow logically and transition smoothly between ideas. Use connecting words and phrases (e.g., "Additionally," "Furthermore," "For instance," "As a result,") to enhance readability.

If the answer cannot be found in the context, say "I don't know." **Do not include unsupported statements or make up information.**

Respond in the following format:
---
Statement 1 [1]. Statement 2 [2, 3].

Citations:
[1] <Page Content 1> //Only the page content of the document
[2] <Page Content 2>
[3] <Page Content 3>
---

Here is an example:

---
Iceland is well-known for its culinary scene. The Baejarins Beztu Pylsur hot dog, made of a mix of lamb, beef, and pork, is one of the most popular street foods [1]. Additionally, classic dishes like fish and chips, Arctic cod, and salmon soup highlight the country's reliance on freshly caught fish from the Arctic sea [1, 2].

Citations:
[1]: "The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger."
[2]: "Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, Arctic cod, and salmon soup!"
---

Question: {question}

Context: {context}

Helpful Answer:
"""


In [132]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
llm = ChatGroq()
llm_pipeline = llm | StrOutputParser()
answer = llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [133]:
answer

"Iceland offers a variety of activities for tourists. For instance, the country is renowned for its unique culinary scene. The Baejarins Beztu Pylsur hot dog, a mix of lamb, beef, and pork, is a popular street food [4]. Additionally, Iceland is very famous for its freshly caught fish from the Arctic sea, with dishes like the classic fish and chips, Arctic cod, and salmon soup being particularly popular [4, 5]. \n\nBeyond its food, Iceland is also a must-visit destination for adventurous people. The country offers numerous opportunities for thrilling activities such as hiking active volcanoes, driving a jeep through volcanic ash, exploring a natural ice cave, and seeing waterfalls [6]. \n\nFurthermore, for those interested in diving, Iceland is home to one of the most famous diving sites in the world, Silfra. This is the only diving site where you can dive between two tectonic plates, and the water is so fresh that you can drink it [7]. \n\nIn terms of transportation, getting around Rey

In [134]:
answer.split('\n')

['Iceland offers a variety of activities for tourists. For instance, the country is renowned for its unique culinary scene. The Baejarins Beztu Pylsur hot dog, a mix of lamb, beef, and pork, is a popular street food [4]. Additionally, Iceland is very famous for its freshly caught fish from the Arctic sea, with dishes like the classic fish and chips, Arctic cod, and salmon soup being particularly popular [4, 5]. ',
 '',
 'Beyond its food, Iceland is also a must-visit destination for adventurous people. The country offers numerous opportunities for thrilling activities such as hiking active volcanoes, driving a jeep through volcanic ash, exploring a natural ice cave, and seeing waterfalls [6]. ',
 '',
 'Furthermore, for those interested in diving, Iceland is home to one of the most famous diving sites in the world, Silfra. This is the only diving site where you can dive between two tectonic plates, and the water is so fresh that you can drink it [7]. ',
 '',
 'In terms of transportation,

<br/>
<br/>
<br/>

## **Method Type 7: Direct Prompting with statement-wise citations**

Not all models support tool calling/function calling or have native JSON mode support. This method explores the use of direct prompting to ask the model to use a specific format

Additionally, we use few-shot prompting to enable in-context learning

### **Simple Experiment Data to test and observe behaviour of Method Type 7**

In [141]:
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'},
    ),
    Document(
        page_content="The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembes a tongue.",
        metadata={'country': 'Norway', 'source': 'norwayhikes', 'link': 'https://www.norwayhikes.com/'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'},
    ),
    Document(
        page_content="Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!",
        metadata={'country': 'Iceland', 'source': 'IcelandGov', 'link': 'https://www.welcometoiceland.com/'},
    ),
    Document(
        page_content="The pasteries and bread in Iceland are fantastic, there are many bakeries in Iceland. One of the most popular bread is called dark rye bread ",
        metadata={'country': 'Iceland', 'source': 'IcelandFood', 'link': 'https://www.icelandicdelicacies.com/'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    ),
    Document(
        page_content="Driving in Iceland is an amazing experience - open roads, majestic volcanos and towering mountains along the way, sheep and arctic foxes make it a great experience. All you need is an international driving license. And, please drive slowly during the winter!",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    ),
    Document(
        page_content="Iceland is a must-go to place for adventurous people! You can hike active volcanoes, drive a jeep through the volcanic ash, explore a natural ice cave, see waterfalls. There are so many opportunities for an adventurer.",
        metadata={'country': 'Iceland', 'source': 'IcelandAdventures', 'link': 'https://www.icelandadventures.com/'},
    ),
    Document(
        page_content="One of the most famous diving sites in the world, Silfra, is located in Iceland! It is the only diving site in the world where you can dive between 2 tectonic plates. The water is also so fresh that you can drink from it, it is the best water that you will ever taste.",
        metadata={'country': 'Iceland', 'source': 'IcelandDiving', 'link': 'https://www.icelanddiving.com/'},
    ),
]

### **Question**

In [165]:
question = "What adventures are there to do in Iceland"

In [170]:
prompt = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
For each statement in your answer, **you must include at least one inline numbered citation** (e.g., [1], [2]) for the Document Objects supporting it.
Always reset the numbering for citations to start from **1** for each response, and ensure the numbering increases sequentially.
Statements without sufficient support from the context should not be included in the answer.

At the bottom, provide the full citations corresponding to each number, but include **only the page content** of the Document Object (exclude metadata).

Write your response in a natural and coherent way, ensuring that the statements flow logically and transition smoothly between ideas. Use connecting words and phrases (e.g., "Additionally," "Furthermore," "For instance," "As a result,") to enhance readability.

If the answer cannot be found in the context, say "I don't know." **Do not include unsupported statements or make up information.**

Respond in the following format:
---
Statement 1 [1]. Statement 2 [2, 3].

Citations:
[1]: <Page Content 1> //Only the page content of the document
[2]: <Page Content 2>
[3]: <Page Content 3>
---

Here are a few examples:

---
Iceland is well-known for its culinary scene, for instance the Baejarins Beztu Pylsur hot dog, made of a mix of lamb, beef, and pork, is one of the most popular street foods [1]. Additionally, classic dishes like fish and chips, Arctic cod, and salmon soup highlight the country's reliance on freshly caught fish from the Arctic sea [1, 2].

Citations:
[1]: "The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger."
[2]: "Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, Arctic cod, and salmon soup!"
---

---
The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga [1, 2]. The Reinebringen hike, although not one of the highest peaks, offers an iconic view of the Reine fjord from its summit [1]. Preikestolen, Kjeragbolten, and Trolltunga are famous for their stunning fjord views and unique geological formations, such as a boulder stuck between a mountain crevasse and a tongue-shaped rock [2].

Citations:
[1]: "The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep."
[2]: "The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembles a tongue."
---



Question: {question}

Context: {context}

Helpful Answer:
"""


In [171]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
llm = ChatGroq()
llm_pipeline = llm | StrOutputParser()
answer = llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [172]:
answer

'Iceland offers a variety of adventures for visitors [5]. For instance, it is a must-go place for adventurous people who enjoy activities such as hiking active volcanoes, driving a jeep through volcanic ash, and exploring natural ice caves [5]. The country also boasts stunning waterfalls [5]. \n\nMoreover, Iceland\'s unique geographical location provides opportunities for unique adventures. One of the most famous diving sites in the world, Silfra, is located in Iceland. This site is the only place where you can dive between two tectonic plates, and the water is so fresh that you can drink from it [6].\n\nTransportation within Reykjavik is convenient with a public bus service, but transportation to places outside Reykjavik requires a car. Some options include car rentals and bus tours [4]. Driving in Iceland is an amazing experience, with open roads, majestic volcanos, and towering mountains along the way [3]. \n\nIn terms of culinary adventures, Iceland\'s famous street food is the Bae

In [173]:
answer.split('\n')

['Iceland offers a variety of adventures for visitors [5]. For instance, it is a must-go place for adventurous people who enjoy activities such as hiking active volcanoes, driving a jeep through volcanic ash, and exploring natural ice caves [5]. The country also boasts stunning waterfalls [5]. ',
 '',
 "Moreover, Iceland's unique geographical location provides opportunities for unique adventures. One of the most famous diving sites in the world, Silfra, is located in Iceland. This site is the only place where you can dive between two tectonic plates, and the water is so fresh that you can drink from it [6].",
 '',
 'Transportation within Reykjavik is convenient with a public bus service, but transportation to places outside Reykjavik requires a car. Some options include car rentals and bus tours [4]. Driving in Iceland is an amazing experience, with open roads, majestic volcanos, and towering mountains along the way [3]. ',
 '',
 "In terms of culinary adventures, Iceland's famous stree

<br/>
<br/>
<br/>
<br/>
<br/>

## **Conclusions**

We decide to use LLMs from LLM providers such as <u>ChatGroq</u> due to the fast inference speed and ability to output well-structured outputs which makes it easy for formatting